## anansi_.marketdata.klines

In [1]:
from anansi_.marketdata.klines import klines_getter
from anansi_.marketdata.models import Market

In [2]:
my_market = Market(broker_name="Binance", ticker_symbol="ETHUSDT", time_frame="2h")


my_klines_getter = klines_getter(my_market)

In [3]:
my_klines = my_klines_getter.newest(1367)

In [5]:
my_klines.apply_indicator.trend.price_from_kline()

In [9]:
class SetupSMA:
    price_metrics = "hlc3"
    number_samples = 20

my_setup = SetupSMA()

my_klines.apply_indicator.trend.simple_moving_average(my_setup)

In [10]:
my_klines

,Open_time,Open,High,Low,Close,Volume,Price,SMA_20
0,2020-07-29 00:00:00,316.77,318.82,312.38,315.23,57152.40884,315.476667,NaN
1,2020-07-29 02:00:00,315.22,317.74,314.36,316.48,41062.98672,316.193333,NaN
2,2020-07-29 04:00:00,316.48,319.40,314.12,318.07,56046.61470,317.196667,NaN
3,2020-07-29 06:00:00,318.05,322.76,317.63,321.11,99787.99796,320.500000,NaN
4,2020-07-29 08:00:00,321.10,325.00,319.25,320.85,107879.82237,321.700000,NaN
...,...,...,...,...,...,...,...,...
1362,2020-11-19 12:00:00,473.47,476.93,472.00,476.24,95450.77545,475.056667,476.035000
1363,2020-11-19 14:00:00,476.21,478.44,473.52,475.52,100173.08296,475.826667,475.802667
1364,2020-11-19 16:00:00,475.51,477.04,473.22,474.76,63567.08465,475.006667,475.263167
1365,2020-11-19 18:00:00,474.74,476.59,471.31,474.10,46355.68576,474.000000,474.655167


In [ ]:
my_market.broker_name

## anansi_.marketdata.brokers

###  Exemplo com a Binance

In [ ]:
from anansi_.marketdata.brokers import Binance

my_broker = Binance()

### Alguns atributos da exchange/broker

In [ ]:
my_broker.base_endpoint, my_broker.DateTimeFmt, my_broker.DateTimeUnit

### Verificando horário do servidor com broker.server_time()

In [ ]:
import pendulum # Só para gerar o formato 'human readable', para exemplo

server_time = dict(server_time_timestamp=my_broker.server_time(),
                   server_time_human_readable=(
                       pendulum.from_timestamp(my_broker.server_time()).to_datetime_string()
                   )
                  )

server_time

### Verificando se o limite de requests foi atingido

In [ ]:
my_broker.was_request_limit_reached()

### Finalmente, solicitando algumas klines

In [ ]:
my_klines = my_broker.get_klines(ticker_symbol="BTCUSDT", time_frame="2h")

my_klines

### Função get_broker

É possível repetir todos os passos acima, desde que haja suporte para o broker (passado para a função 'get_broker' pelo parâmetro 'broker_name') instanciando um broker como mostrado abaixo (novamente tomando a binance como exemplo):

In [ ]:
from anansi_.marketdata.brokers import get_broker

my_broker = get_broker(broker_name="binance")

Testando com as klines; esteja livre para repetir os passos anteriores :)

In [ ]:
my_klines = my_broker.get_klines(ticker_symbol="BTCUSDT", time_frame="2h")

my_klines

In [ ]:
type(my_klines)